In [ ]:
# Author: Pavan Kumar Raja

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# os.makedirs('drive/MyDrive/NLP_PROJECT/')

In [ ]:
os.chdir('drive/MyDrive/NLP_PROJECT/')

In [ ]:
# T0pp is too big to download and use 48GB, T0_3B don't give good performance.

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
# model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")

In [ ]:
# import random

In [ ]:
# sentence_tokens = ["weights", "rope", "pully", "man", "women"]
# for i in range(5):
#     inputs = tokenizer.encode(f"Ask two questions on topic {random.choice(sentence_tokens)}, Output generated questions as Question1, Question2", return_tensors="pt")
#     outputs = model.generate(inputs)
#     for x in outputs:
#         print(tokenizer.decode(x))

In [ ]:
# My api keys are revoked.
!pip install banana-dev

In [ ]:
import csv
import banana_dev as banana
banana_api = {Banana API}

In [ ]:
import csv
import os
from tqdm import tqdm
import banana_dev as banana
from difflib import SequenceMatcher

# Limiting hypothesis to maximum 5 sentences, in case there is a break before complition of a sentence.
def process_sentences(sentence):
    if sentence.endswith('.'): return sentence
    sentences = sentence.split('.')
    if len(sentences) == 1:
        sentence = sentences[0]
    else:
        sentences = sentences[:-1]
        sentences = sentences[:5]
        sentence = sentences[0] + '.'.join(sentences[1:]) + '.'
    return sentence

# Premise hypothesis - text
def premise_hypothesis_normal():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a premise.",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])

    model_parameters = {
        "text": f"Generatec a hypothesis form the premise. Premise: {sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    return (sentence1, sentence2, 'entailment')

# Premise hypothesis - conversation
def premise_hypothesis_conversation():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a premise in conversation.",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate a hypothesis form the premise. Premise:{sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    return (sentence1, sentence2, 'entailment')

# Premise hypothesis neutral
def premise_hypothesis_neutral_normal():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a premise",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate a random sentence from premise:{sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    return (sentence1, sentence2, 'neutral')

# Premise hypothesis neutral
def premise_hypothesis_neutral_conversation():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a premise in conversation.",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate a random sentence from premise:{sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    return (sentence1, sentence2, 'neutral')

# Premise hypothesis contradiction
def premise_hypothesis_contradict_normal():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a premise.",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate hypothesis from the premise. Premise: {sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate a contradictory sentence for : {sentence2}",
        "max_length": 64,
    }
    out3 = banana.run(banana_api, model_key, model_parameters)
    sentence3 = out3['choices'][-1]['output'][-1]
    return (sentence1, sentence2, sentence3, 'contradiction')\

def premise_hypothesis_contradict_conversation():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a random fact:",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = process_sentences(out1['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate hypothesis from the premise. Premise: {sentence1}",
        "max_length": 64,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    sentence2 = process_sentences(out2['choices'][-1]['output'][-1])
    model_parameters = {
        "text": f"Generate a contradictory sentence for: {sentence2}",
        "max_length": 64,
    }
    out3 = banana.run(banana_api, model_key, model_parameters)
    sentence3 = out3['choices'][-1]['output'][-1]
    return (sentence1, sentence2, sentence3, 'contradiction')


def main():
    write_header = True
    QQP_header = ['Premise', 'Hypothesis', 'Label']
    QQP_csv_filename = "premise_hypo_dataset.csv"
    if os.path.exists(QQP_csv_filename):
        write_header = False
    with open(QQP_csv_filename, 'a', newline='') as csv_file_descriptor:
        csv_writer = csv.writer(csv_file_descriptor)
        if write_header:
            csv_writer.writerow(QQP_header)
        for _ in tqdm(range(1000)):
            s1, s2, state = premise_hypothesis_normal()
            if SequenceMatcher(None, s1, s2).ratio() != 1.0:
                csv_writer.writerow([s1, s2, state])
            s1, s2, state = premise_hypothesis_conversation()
            if SequenceMatcher(None, s1, s2).ratio() != 1.0:
                csv_writer.writerow([s1, s2, state])
            s1, s2, state = premise_hypothesis_neutral_normal()
            if SequenceMatcher(None, s1, s2).ratio() != 1.0:
                csv_writer.writerow([s1, s2, state])
            s1, s2, state = premise_hypothesis_neutral_conversation()
            if SequenceMatcher(None, s1, s2).ratio() != 1.0:
                csv_writer.writerow([s1, s2, state])
            s1, s2, s3, state = premise_hypothesis_contradict_normal()
            if SequenceMatcher(None, s2, s3).ratio() != 1.0:
                csv_writer.writerow([s1, s3, state])
            s1, s2, s3, state = premise_hypothesis_contradict_conversation()
            if SequenceMatcher(None, s2, s3).ratio() != 1.0:
                csv_writer.writerow([s1, s3, state])
main()

In [ ]:
# Question question quora.

def generate_q_q_pair():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a random long question.",
    "max_length": 256,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    q1 = out1['choices'][-1]['output'][-1]
    model_parameters = {
        "text" : f"Paraphrase: {q1}",
        "max_length": 256,
    }
    out2 = banana.run(banana_api, model_key, model_parameters)
    q2 = out2['choices'][-1]['output'][-1]
    model_parameters = {
        "text" : f'Generate a question on: {q2}',
        "max_length": 256,
    }
    out3 = banana.run(banana_api, model_key, model_parameters)
    q3 = out3['choices'][-1]['output'][-1]

    return (q1, q3)

def generate_random_question_pair():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a random question.",
    "max_length": 120,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    q1 = out1['choices'][-1]['output'][-1]
    out2 = banana.run(banana_api, model_key, model_parameters)
    q2 = out2['choices'][-1]['output'][-1]
    return (q1, q2)

In [ ]:
generate_q_q_pair()

In [ ]:
!pip install tdqm

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# QQ header
def generate_question_question_dataset():
    write_header = True
    QQP_header = ['question1', 'question2', 'is_duplicate']
    QQP_csv_filename = "Question_Question_dataset.csv"
    if os.path.exists(QQP_csv_filename):
        write_header = False
    with open(QQP_csv_filename, 'a', newline='') as csv_file_descriptor:
        csv_writer = csv.writer(csv_file_descriptor)
        if write_header:
            csv_writer.writerow(QQP_header)
        for i in tqdm(range(10000)):
            q1, q2 = generate_q_q_pair()
            if q1 and q2 and q2.endswith('?'):
                csv_writer.writerow([q1, q2, True])
            q3, q4 = generate_random_question_pair()
            if q3 and q4: 
                csv_writer.writerow([q3, q4, False])
    print("Kekl question answer done!!!")

generate_question_question_dataset()

In [ ]:
# gramatical and ungramatical Gpt3 davince.
# Tried T0_pp and GPT-3 davince, wasn;t able to generate ungrammatical sentences.

In [ ]:
def generate_correct_incorrect_pairs():
    model_key = "t0pp"
    model_parameters = {
    "text": "Generate a sentences with comma seperation.",
    "max_length": 120,
    }
    out1 = banana.run(banana_api, model_key, model_parameters)
    sentence1 = out1['choices'][-1]['output'][-1]
    return (sentence1, sentence2)

In [ ]:
print(generate_correct_incorrect_pairs())

In [ ]:
from transformers import pipeline
generator = pipeline("text-generation")

In [ ]:
generator("generator related sentences with a comma separator.") # Clearly didn't work

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'generator related sentences with a comma separator. You may also use a space.\n\nExample #2:\n\na.foo{...}\n\nand c.foo{foo}\n\nexample #3:\n\na.'}]

In [ ]:
def clean_string(s):
    if not s: return s
    new_s = s.split('\n')
    s_list = []
    for x in new_s:
        if x.endswith('.'): s_list.append(x)
        elif len(x.split('.')) > 1:
            s_list.append('.'.join(x.split('.')[:-1]) + '.')
        else:
            s_list.append(x + '.')
    new_s = s_list
    if new_s[0][0].islower():
        new_s[0] = new_s[0][0].upper() + new_s[0][1:]
    return ' '.join(set(new_s))

In [ ]:
!pip install openai

In [ ]:
import os
import openai

openai.api_key = {Open-ai-key}

def generate_with_gpt3(engine, prompt):
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=0.60,
        max_tokens=32,
        top_p=0.75,
        frequency_penalty=0.75,
        presence_penalty=0.5
    )
    return clean_string(response.choices[0].text)

In [ ]:
# Generate all sentences using gpt3 ada, we can iter later using the the text file.
from tqdm.notebook import tqdm
with open("sentence_list_2.txt", 'a') as fd:
    for _ in tqdm(range(10000)):
        text = generate_with_gpt3('davinci-instruct-beta-v3', "Generate a long sentences on interesting topic.")
        if text:
            new_text = generate_with_gpt3('davinci-instruct-beta-v3', f"Given a sentence, introduce grammatical error. Sentece:{text}")
            fd.write(new_text + '\n')
    
    # Changed different prompts it didn't work.
    # for _ in tqdm(range(2000)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Generate a cause effect sentence.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(200)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Generate a sentence with random mood.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(5000)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Give a sentence from wikipedia.")
    #     if text:
    #         fd.write(text + '\n') 
    # for _ in tqdm(range(3000)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Narrate a random incident.")
    #     if text:
    #         fd.write(text + '\n') 
    # for _ in tqdm(range(8000)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Give a complex sentence.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(500)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Give a long sentence with a lot of expression.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(500)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Summarise a random historical event.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(500)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Describe some random event.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(200)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Give a conclusion to random story.")
    #     if text:
    #         fd.write(text + '\n')
    # for _ in tqdm(range(200)):
    #     text = generate_with_gpt3('davinci-instruct-beta-v3', "Describe a day in random profession.")
    #     if text:
    #         fd.write(text + '\n')
    

In [ ]:
# Using text generated for QQ_pair and MNLI dataset, I'm manualy introducing the errors.
# Ran this on local machine.

In [ ]:
import random
import os
import csv
from tqdm import tqdm


tense_list = "awake awoke awoken be were been bear bore born beat beat beat become became become begin began begun " \
             "bend bent bent beset beset beset bet bet bet bid bade bid bind bound bound bite bit bitten bleed bled " \
             "bled blow blew blown break broke broken breed bred bred bring brought brought broadcast broadcast " \
             "broadcast build built built burn burned burned burst burst burst buy bought bought cast cast cast catch " \
             "caught caught choose chose chosen cling clung clung come came come cost cost cost creep crept crept cut " \
             "cut cut deal dealt dealt dig dug dug dive dove dived do did done draw drew drawn dream dreamt dreamed " \
             "drive drove driven drink drank drunk eat ate eaten fall fell fallen feed fed fed feel felt felt fight " \
             "fought fought find found found fit fit fit flee fled fled fling flung flung fly flew flown forbid " \
             "forbade forbidden forget forgot forgotten forgo foregone foregone forgive forgave forgiven forsake " \
             "forsook forsaken freeze froze frozen get got gotten give gave given go went gone grind ground ground " \
             "grow grew grown hang hung hung hear heard heard hide hid hidden hit hit hit hold held held hurt hurt " \
             "hurt keep kept kept kneel knelt knelt knit knit knit know knew know lay laid laid lead led led leap " \
             "leaped leapt learn learned learned leave left left lend lent lent let let let lie lay lain light lit " \
             "lighted lose lost lost make made made mean meant meant meet met met misspell misspelled misspelled " \
             "mistake mistook mistaken mow mowed mown overcome overcame overcome overdo overdid overdone overtake " \
             "overtook overtaken overthrow overthrew overthrown pay paid paid plead pled pled prove proved proven put " \
             "put put quit quit quit read read read rid rid rid ride rode ridden ring rang rung rise rose risen run " \
             "ran run saw sawed sawn say said said see saw seen seek sought sought sell sold sold send sent sent set " \
             "set set sew sewed sewn shake shook shaken shave shaved shaven shear shore shorn shed shed shed shine " \
             "shone shone shoe shoed shod shoot shot shot show showed shown shrink shrank shrunk shut shut shut sing " \
             "sang sung sink sank sunk sit sat sat sleep slept slept slay slew slain slide slid slid sling slung " \
             "slung slit slit slit smite smote smitten sow sowed sown speak spoke spoken speed sped sped spend spent " \
             "spent spill spilt spilled spin spun spun spit spat spit split split split spread spread spread spring " \
             "sprang sprung stand stood stood steal stole stolen stick stuck stuck sting stung stung stink stank " \
             "stunk stride strod stridden strike struck struck string strung strung strive strove striven swear swore " \
             "sworn sweep swept swept swell swelled swelled swim swam swum swing swung swung take took taken teach " \
             "taught taught tear tore torn tell told told think thought thought thrive thrived/throve thrived throw " \
             "threw thrown thrust thrust thrust tread trod trodden understand understood understood uphold upheld " \
             "upheld upset upset upset wake woke woken wear wore worn weave weaved woven wed wed wed weep wept wept " \
             "wind wound wound win won won withhold withheld withheld withstand withstood withstood wring wrung wrung " \
             "write wrote written "
tense = zip(*(iter(tense_list.split(" ")),) * 3)
tense_dict = dict()
for x, y, z in tense:
    tense_dict[x] = [y, z]
    tense_dict[y] = [x, z]
    tense_dict[z] = [x, y]
tense_set = set(tense_dict.keys())

articles = ['a', 'an', 'the']

random_character = "! & _ - : ; ? / . , ' \"".split()

pronouns_table = ["I me mine my myself", "you yours your yourself", "she her hers herself", "he his him himself",
                  "they them their theirs themself", "it its itself", "we us ours our ourselves", "one one's oneself"]
pronoun_dict = dict()
for pronouns in pronouns_table:
    pronoun_list = pronouns.split(' ')
    for i in range(len(pronoun_list)):
        pronoun_dict[pronoun_list[i]] = pronoun_list[:i] + pronoun_list[i+1:]
pronoun_keys = set(pronoun_dict.keys())

singular_plural_verb_pairs = "agrees agree bakes bake calls call counts count drops drop enjoys enjoy forces force " \
                             "invents invent makes make points point pulls pull stirs stir uses use waves wave " \
                             "attaches attach catches catch crashes crash fixes fix guesses guess itches itch kisses " \
                             "kiss pitches pitch pushes push taxes tax teaches teach waltzes waltz washes wash wishes " \
                             "wish annoys annoy brays bray buys buy delays delay employes employ lays lay obeys obey " \
                             "pays pay plays play prays pray says say sprays spray stays stay sways sway applies " \
                             "apply bullies bully carries carry cries cry defies defy dries dry flies fly hurries " \
                             "hurry marries marry pitties pity replies reply spies spy try tries worries worry "
singular_plural_verb_pairs_list = zip(*(iter(singular_plural_verb_pairs.split(" ")),) * 2)
singular_plural_verb_dict = dict()

for x, y in singular_plural_verb_pairs_list:
    singular_plural_verb_dict[x] = y
    singular_plural_verb_dict[y] = x
singular_plural_verb_set = set(singular_plural_verb_dict.keys())

singular_plural_noun_pairs = "Army-Armies Ass-Asses Baby-Babies Bamboo-Bamboos Bench-Benches Bird-Birds Boat-Boats " +\
                             "Bone-Bones Box-Boxes Boy-Boys Buffalo-Buffaloes Bus-Buses Bush-Bushes Caddy-Caddies " +\
                             "Calf-Calves Car-Cars Cat-Cats Chair-Chairs Chief-Chiefs Child-Children City-Cities " +\
                             "Class-Classes Class fellow-Class fellows Cliff-Cliffs Clutch-Clutches Copy-Copies " +\
                             "Country-Countries Cow-Cows Cry-Cries Cuckoo-Cuckoos Cup-Cups Day-Days Deci-Decies " +\
                             "Deer-Deers Dog-Dogs Donkey-Donkeys Dozen-Dozens Duty-Duties Essay-Essays " +\
                             "Family-Families Father-Fathers Fish-Fishes Fly-Flies Foot-Feet Fox-Foxes Gas-Gases " +\
                             "Glass-Glasses Grass-Grasses Hair-Hairs Half-Halves Hand-Hands Hero-Heroes Hoof-Hoofs " +\
                             "Horse-Horses House-Houses Inch-Inches Jar-Jars Key-Keys Knife-Knives " +\
                             "Lady-LadiesLass-Lasses Leaf-Leaves Leg-Legs Life-Lives Loaf-Loaves Loof-Loofs " +\
                             "Love-Loves Maidservant-Maidservants Man-Men Mango-Mangoes Monkey-Monkeys MotherMothers " +\
                             "Mouse- Mice News-News Ox-Oxen Pencil-Pencils Penny-Pennies Person-People Pitch-Pitches " +\
                             "Poetry-Poetries Potato-Potatoes Proof-Proofs Quiz-Quizzes Radio-Radios Ray-Rays " +\
                             "River-River  Scissor-Scissors Self-Selves Sheep-Sheeps Shop-Shops Sister-Sisters " +\
                             "Sky-Skies Spey- Spies Stepson-Stepsons Story-Stories Table-Tables Thief-Thieves " +\
                             "Tooth-Teeth Toy-Toys Trouser-Trousers Uncle-Uncles Watch-Watches Wife-Wives Wish-Wishes " +\
                             "Woman-Women"
singular_plural_noun_pairs_list = [x.split("-") for x in singular_plural_noun_pairs.split(" ") if len(x.split("-")) == 2]
singular_plural_noun_dict = dict()
for x in singular_plural_noun_pairs_list:
    singular_plural_noun_dict[x[0]] = x[1]
    singular_plural_noun_dict[x[1]] = x[0]

singular_plural_noun_set = set(singular_plural_verb_dict.keys())


def change_tense(s):
    words = [x for x in s.split(' ') if x in tense_set]
    choices = random.randint(0, len(words)+1 // 2)
    if not words or not choices: return s
    words = random.choices(words, k = choices)
    t = s
    for w in words:
        t = t.replace(' '+w+' ', ' '+random.choice(tense_dict[w])+' ', 2)
    return t


def change_articles(s):
    t = s
    article_count = len([x for x in s.split(' ') if x in articles])
    for _ in range(random.randint(0, article_count)):
        t = t.replace(' '+random.choice(articles)+' ', ' ' + random.choice(articles) + ' ')
    return t


def change_to_random_symbol(s):
    t = s
    characters = list(set([x for x in s if x in random_character]))
    choices = random.randint(0, len(characters)+1 // 2)
    if not characters or not choices: return s
    characters = random.choices(characters, k = choices)
    for c in characters:
        t = t.replace(' '+c+' ', ' '+random.choice(random_character)+' ', 1)
    return t


def replace_pronoun(s):
    t = s
    s_pronoun = list(set([x for x in s.split(' ') if x in pronoun_keys]))
    choices = random.randint(0, len(s_pronoun)+1 // 2)
    if not s_pronoun or not choices: return s
    s_pronoun = random.choices(s_pronoun, k = choices)
    for pn in s_pronoun:
        t = t.replace(' '+pn+' ', ' '+random.choice(pronoun_dict[pn])+' ', 1)
    return t


def switch_singular_plural_verb(s):
    t = s
    word_list = list(set([x for x in s.split(' ') if x in singular_plural_verb_set]))
    choices = random.randint(0, len(word_list)+1 //2)
    if not word_list or not choices: return s
    word_list = random.choices(word_list, k = choices)
    for w in word_list:
        t = t.replace(w, ' '+singular_plural_verb_dict[w]+' ', 1)
    return t


def shuffle_words(s, max_shuffle=2):
    s_list = s.split(' ')
    for _ in range(max_shuffle):
        i, j = random.choices(range(0, len(s_list) - 1), k=2)
        s_list[i], s_list[j] = s_list[j], s_list[i]
    return ' '.join(s_list)


def make_everything_caps(s):
    return s.upper()


def make_everything_lower(s):
    return s.lower()


def remove_space_after_fullstop(s):
    choices = random.randint(0, 4)
    return s.replace('. ', '.', choices)


def make_first_lower(s):
    return s[0].lower() + s[1:]


def change_fullstop_comma(s):
    choices = random.randint(0, 2)
    return s.replace('.', ',', choices)


def change_comma_fullstop(s):
    choices = random.randint(0, 2)
    return s.replace(',', '.', choices)


def remove_apostrophe(s):
    choices = random.randint(0, 2)
    return s.replace("'", '', choices)


def remove_commas(s):
    choices = random.randint(0, 2)
    return s.replace(',', '', choices)


def remove_fullstop(s):
    choices = random.randint(0, 2)
    return s.replace('.', '', choices)


function_list = [remove_fullstop, remove_commas, remove_apostrophe, change_comma_fullstop, change_fullstop_comma,
                 make_first_lower, remove_space_after_fullstop, make_everything_lower, make_everything_caps,
                 shuffle_words, switch_singular_plural_verb, replace_pronoun, change_to_random_symbol, change_articles,
                 change_tense]

MAX_CHOICES = 6
HEADER = ['acceptability', 'sentence']
CSV_FILE = "grammer.csv"

with open('mix_sentence.txt', 'r') as fr:
    write_header = True
    edit_sentence = 1
    if os.path.exists(CSV_FILE): write_header = False
    with open('grammer.csv', 'a', newline='') as fw:
        csv_writer = csv.writer(fw)
        if write_header:
            csv_writer.writerow(HEADER)
        sentences = fr.readlines()
        for sen in tqdm(range(len(sentences))):
            my_sen = sentences[sen]
            if not edit_sentence:
                csv_writer.writerow([my_sen.rstrip(), edit_sentence])
                edit_sentence = 1
            else:
                fun_choices = random.randint(2, MAX_CHOICES)
                my_funcs = random.sample(function_list, k=fun_choices)
                for fun in my_funcs:
                    my_sen = fun(my_sen)
                csv_writer.writerow([my_sen.rstrip(), edit_sentence])
                edit_sentence = 0


In [ ]:
#### Aggregated utils used so far

import os
import csv
import hashlib
from difflib import SequenceMatcher
from langdetect import detect
import random


def count_unique(csv_file):
    unique_set = set()
    with open(csv_file, 'r', newline='') as csv_file_descriptor:
        csv_reader = csv.reader(csv_file_descriptor)
        next(csv_reader, None)  # skip the headers
        count = 0
        for [q1, q2, state] in csv_reader:
            count += 2
            unique_set.add(hashlib.sha256(q1.encode()))
            unique_set.add(hashlib.sha256(q2.encode()))
    
    print(count, len(unique_set))

# Extarct all both csv and make ungrammatic dataset
def get_sentences():
    endings = set('."\'?!')
    with open("mix_sentence.txt", 'a') as fw:
        line_list = []
        with open("premise_hypo_dataset.csv", 'r', newline='') as hypo_csv:
            reader = csv.reader(hypo_csv)
            for [p, h, _] in reader:
                if len(p.split()) > 8:
                    if p[-1] not in endings: p+='.'
                    line_list.append(p[0].upper() + p[1:]+'\n')
                if len(h.split()) > 8:
                    if h[-1] not in endings: h += '.'
                    line_list.append(h[0].upper() + h[1:]+'\n')
        with open("Question_Question_dataset.csv", 'r', newline='') as question_csv:
            reader = csv.reader(question_csv)
            for [p, h, _] in reader:
                if len(p.split()) > 8:
                    if p[-1] not in endings: p += '.'
                    line_list.append(p[0].upper() + p[1:]+'\n')
                if len(h.split()) > 8:
                    if h[-1] not in endings: h += '.'
                    line_list.append(h[0].upper() + h[1:]+'\n')
        random.shuffle(line_list)
        fw.writelines(line_list)

def filter_same_hypothesis_premise(csv_file, new_csv_file):
    with open(csv_file, 'r', newline='') as base_csv:
        with open(new_csv_file, 'a', newline='') as new_csv:
            reader = csv.reader(base_csv)
            writer = csv.writer(new_csv)
            for [p, h, label] in reader:
                if p and h and (SequenceMatcher(None, p, h).ratio() != 1.0):
                    writer.writerow([p, h, label])


# Code to filter duplicates.
def copy_unique(f1 ,f2):
    count = 0
    unique_set = set()
    with open(f1, 'r') as fr:
        with open(f2, 'a') as fw:
            for line in fr.readlines():
                line.replace('. .', '.')
                count += 1
                hash_k = hashlib.sha256(line.encode()).hexdigest()
                if hash_k in unique_set: continue
                unique_set.add(hash_k)
                try:
                    if detect(line) == 'en':
                        fw.write(line)
                except:
                    continue
    print(count, len(unique_set))


# Code to remove bad line endings
def remove_bad_line_endings(f1, f2):
    with open(f1, 'r') as fr:
        with open(f2, 'w') as fw:
            count = 0
            for line in fr.readlines():
                count += 1
                write_line = line.lstrip()
                write_line = write_line.rstrip() + '\n'
                fw.write(write_line)
    print(count)


def QQ_restructure_to_orginal_dataset():
    row_id = 0
    question_count = 0
    OLD_CSV = "Question_Question_dataset.csv"
    NEW_CSV = "MY_QQP_dataset.csv"
    NEW_HEADER = ['id', 'qi1', 'qid2', 'question1', 'question2', 'is_duplicate']
    with open(NEW_CSV, 'a') as fw:
        csv_writer = csv.writer(fw)
        csv_writer.writerow(NEW_HEADER)
        with open(OLD_CSV, 'r') as fr:
            csv_reader = csv.reader(fr)
            csv_reader.__next__()
            for [q1, q2, is_d] in csv_reader:
                d = 1 if is_d == 'True' else 0
                csv_writer.writerow([row_id, question_count, question_count+1, q1, q2, d])
                row_id += 1
                question_count += 2

70086 70086
